# Installations:

In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# Imports:

In [ ]:
import os
import torch
from torch_geometric.data import Data

In [ ]:
import argparse
import time
from tqdm import tqdm
import copy as cp
import torch.nn.functional as F
from torch.utils.data import random_split
from torch_geometric.data import DataLoader, DataListLoader
from torch_geometric.nn import DataParallel
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool, GATConv

# Loading the inputs:

In [ ]:
data_dirs = ['twitter15', 'twitter16']

label_file_path = 'sentiments.txt' #'label.txt'
source_tweets_file_path = 'source_tweets.txt'
tree_file_dir = 'tree'
root_path = '/content/drive/MyDrive/retweet_cascade'

num_features = 1

graph_list = []
labels = []


for data_dir in data_dirs:
    dir_path = os.path.join(root_path, data_dir)

    for file_name in os.listdir(os.path.join(dir_path, tree_file_dir)):
        if not file_name.endswith('.txt'):
            continue

        tweet_id = file_name[:-4]
        if len(tweet_id) > 18:
                      print(tweet_id[:18])
                      tweet_id = tweet_id[:18]

        edge_list = []
        with open(os.path.join(dir_path, tree_file_dir, file_name)) as f:
            for line in f:
                parent, child = line.strip().split('->')
                parent, child = parent.replace("'", "").strip(), child.replace("'", "").strip()
                parent = parent.replace("ROOT", str(tweet_id))

                edge_list.append([int(parent.split(',')[1]), int(child.split(',')[1])])

        label = -1
        with open(os.path.join(dir_path, label_file_path)) as f:
            for line in f:
                if line.startswith(tweet_id):
                    label = int(line.strip().split(':')[1])
                    labels.append(label)
                    break

        # features = torch.zeros((1, num_features))
        # with open(os.path.join(dir_path, source_tweets_file_path)) as f:
        #     for line in f:
        #         if line.startswith(tweet_id):
        #             cols = line.strip().split('\t')
        #             features[0, 0] = int(cols[1])  # number of followers
        #             features[0, 1] = int(cols[2])  # number of friends
        #             features[0, 2] = float(cols[3])  # ratio of followers and friends
        #             features[0, 3] = int(cols[4])  # number of history tweets
        #             features[0, 4] = int(cols[5])  # registration time (year)
        #             features[0, 5] = int(cols[6])  # whether verified account or not
        #             break
        features = torch.ones((1, num_features))

        edge_index = torch.tensor(edge_list).t().contiguous()
        x = features.repeat(len(edge_list) + 1, 1)
        data = Data(x=x, edge_index=edge_index, y=label)

        graph_list.append(data)

688096910217879554
688437279405305856
688110059553861634
688281442162352128
688441130271289350
688147918113538049
688502260779843587
688461902821855232


In [ ]:
print(graph_list[0])

Data(x=[138, 1], edge_index=[2, 137], y=1)


In [ ]:
torch.save(graph_list, 'retweet_cascade_dataset.pt')

# Craeting the Dataset:

In [ ]:
# from torch_geometric.data import Data

# epoch_len = 120
# land_mark_num = 18
# len = df.shape[0]-1
# overlap = 0.5
# graph_list = []
# order = []
# count = 0
# step = 10

# #edge_index, edge_attr = set_edges(epoch_len, land_mark_num)

# for i in range(1, len-epoch_len, step):
#   x = set_nodes(norm_Xs, norm_Ys, CLs, i, land_mark_num, epoch_len)
#   data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
#   graph_list.append(data)
#   count = count + 1
#   order.append((count, i))

# graph_num = count

* Saving the dataset:


In [ ]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

loader = DataLoader(graph_list, batch_size=16)
torch.save(loader, '/content/drive/MyDrive/retweets_loader.pth')

In [ ]:
import os.path as osp

import torch
from torch_geometric.data import Dataset, download_url

root = '/content/drive/MyDrive/Test_loader.pth'

class Dataset1(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

# Creating the model:

* some more imports:

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, VGAE
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GAE, VGAE, GCNConv
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import global_mean_pool

from torch_geometric.data import DataLoader
from torch_geometric.datasets import Planetoid

from sklearn.metrics import f1_score

In [ ]:
class GCNSentiment(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, concat=True):
      super(GCNSentiment, self).__init__()

      self.num_features = input_dim
      self.num_classes = output_dim
      self.nhid = hidden_dim
      self.concat = concat

      self.conv1 = GATConv(self.num_features, self.nhid * 2)
      self.conv2 = GATConv(self.nhid * 2, self.nhid * 2)

      self.fc1 = Linear(self.nhid * 2, self.nhid)

      if self.concat:
        self.fc0 = Linear(self.num_features, self.nhid)
        self.fc1 = Linear(self.nhid * 2, self.nhid)
      self.fc2 = Linear(self.nhid, self.num_classes)

    def forward(self, data):
      x, edge_index, batch = data.x, data.edge_index, data.batch

      x = F.selu(self.conv1(x, edge_index))
      x = F.selu(self.conv2(x, edge_index))
      x = F.selu(global_mean_pool(x, batch))
      x = F.selu(self.fc1(x))
      x = F.dropout(x, p=0.5, training=self.training)

      if self.concat:
        news = torch.stack([data.x[(data.batch == idx).nonzero().squeeze()[0]] for idx in range(data.num_graphs)])
        news = F.relu(self.fc0(news))
        x = torch.cat([x, news], dim=1)
        x = F.relu(self.fc1(x))

      x = F.log_softmax(self.fc2(x), dim=-1)
      return x

# Training the model:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_dim = 1
hidden_dim = 16
output_dim = 3
batch_size = 16
epoch_num = 100

# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

dataloader = DataLoader(graph_list, batch_size=batch_size, shuffle=True)

model = GCNSentiment(input_dim, hidden_dim, output_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


model.train()
for epoch in range(epoch_num):
    total_loss = 0
    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = F.nll_loss(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    print("Epoch ", epoch, " loss: ", total_loss / len(graph_list)) # note!


model.eval()
pred_y = []
true_y = []

for batch in dataloader:
    batch = batch.to(device)
    with torch.no_grad():
        out = model(batch.x, batch.edge_index)
        pred = out.argmax(dim=-1)
        pred_y.append(pred.cpu().numpy())
        true_y.append(batch.y.cpu().numpy())

pred_y = np.concatenate(pred_y, axis=0)
true_y = np.concatenate(true_y, axis=0)

f1 = f1_score(true_y, pred_y, average='macro')
print("F1 score: ", f1)

# Let's watch the model!

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.manifold import TSNE

In [ ]:
def get_embeddings(dataset):
    model.eval()
    with torch.no_grad():
        data = dataset[0]
        embeddings = model.conv2(F.selu(model.conv1(data.x, data.edge_index)), data.edge_index)
        return embeddings.cpu().numpy()


dataset = dataset 
last_layer_embeddings = get_embeddings(dataset)


tsne = TSNE(n_components=2)
embeddings_2d = tsne.fit_transform(last_layer_embeddings)

colors = ['red', 'green', 'blue']
labels = labels
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels, cmap=plt.colors.ListedColormap(colors))
plt.show()